In [1]:
import requests

In [2]:
url = 'https://i.pinimg.com/originals/e0/3d/5b/e03d5b812b2734826f76960eca5b5541.jpg'

res = requests.get(url)

In [3]:
res.status_code

200

The HTTP 200 OK success status response code indicates that the request has succeeded. A 200 response is cacheable by default. 

res.content gives binary stuff

In [4]:
f = open("panda.jpg","wb")

f.write(res.content)

f.close()

In [5]:
import requests
from bs4 import BeautifulSoup

In [6]:
url = 'https://www.goodreads.com/quotes/tag/{}'   #?page={}'
# Putting a place holder there. Before it was inspiration

In [7]:
url = url.format('inspiration')

In [8]:
res = requests.get(url)

In [9]:
res.content

b'<!DOCTYPE html>\n<html class="desktop withSiteHeaderTopFullImage\n">\n<head>\n  <title>Inspiration Quotes (12934 quotes)</title>\n\n<meta content=\'12934 quotes have been tagged as inspiration: Eleanor Roosevelt: \xe2\x80\x98Do one thing every day that scares you.\xe2\x80\x99, Kurt Vonnegut: \xe2\x80\x98We are what we pretend to be,...\' name=\'description\'>\n<meta content=\'telephone=no\' name=\'format-detection\'>\n<link href=\'https://www.goodreads.com/quotes/tag/inspiration\' rel=\'canonical\'>\n\n\n\n    <script type="text/javascript"> var ue_t0=window.ue_t0||+new Date();\n </script>\n  <script type="text/javascript">\n    var ue_mid = "A1PQBFHBHS6YH1";\n    var ue_sn = "www.goodreads.com";\n    var ue_furl = "fls-na.amazon.com";\n    var ue_sid = "240-4801966-7246632";\n    var ue_id = "457SWS4H8PHZ3M1C2G99";\n\n    (function(e){var c=e;var a=c.ue||{};a.main_scope="mainscopecsm";a.q=[];a.t0=c.ue_t0||+new Date();a.d=g;function g(h){return +new Date()-(h?0:a.t0)}function d(h){re

In [10]:
#Now we need a parser as it is not machine/human readable

soup = BeautifulSoup(res.text)

In [11]:
soup.p  
#First para

<p>Welcome back. Just a moment while we sign you in to your Goodreads account.</p>

In [12]:
links = soup.find_all("a")
#List of all anchor tags or links

In [13]:
#Divs that contain quotes
quote_divs = soup.find_all('div', attrs={'class' : 'quote'})

In [14]:
quote_div = quote_divs[0]

In [15]:
#Div that contains text in that div (quote_div)
quoteText_div = quote_div.find_next('div', attrs={'class':'quoteText'})

In [16]:
#Empty space 'striped'
striped = quoteText_div.text.strip()

In [17]:
striped_lines = striped.split('\n')

In [18]:
quote = striped_lines[0][1:-1]
author = striped_lines[-1].strip()

In [19]:
print(quote)
print(author)

Do one thing every day that scares you.
Eleanor Roosevelt


In [20]:
left_div = quote_div.find_next('div', attrs={'class' : 'left'})

In [21]:
left_div.find_all("a")

[<a href="/quotes/tag/attributed">attributed</a>,
 <a href="/quotes/tag/fear">fear</a>,
 <a href="/quotes/tag/inspiration">inspiration</a>]

In [22]:
tags = [tag.text for tag in left_div.find_all("a")]

In [23]:
tags

['attributed', 'fear', 'inspiration']

In [24]:
# Now lets try to loop for all right from quote_div
res = requests.get(url)
soup = BeautifulSoup(res.text)

quote_divs = soup.find_all('div', attrs={'class' : 'quote'})

quotes = []

for quote_div in quote_divs:
  quoteText_div = quote_div.find_next('div', attrs={'class':'quoteText'})
  
  striped = quoteText_div.text.strip()
  striped_lines = striped.split('\n')
  
  quote = striped_lines[0][1:-1]
  author = striped_lines[-1].strip()
  
  left_div = quote_div.find_next('div', attrs={'class' : 'left'})
  tags = [tag.text for tag in left_div.find_all("a")]

  quote_item = {
      'quote' : quote,
      'author' : author,
      'tags' : tags
  }

  quotes.append(quote_item)

In [25]:
quotes

[{'author': 'Eleanor Roosevelt',
  'quote': 'Do one thing every day that scares you.',
  'tags': ['attributed', 'fear', 'inspiration']},
 {'author': '//]]>',
  'quote': 'We are what we pretend to be, so we must be careful about what we pretend to be.',
  'tags': ['action',
   'attitude',
   'choice',
   'imagination',
   'inspiration',
   'life',
   'pretend']},
 {'author': '//]]>',
  'quote': 'When we love, we always strive to become better than we are. When we strive to become better than we are, everything around us becomes better too.',
  'tags': ['hope', 'inspiration', 'love', 'santiago']},
 {'author': '//]]>',
  'quote': 'Sometimes you wake up. Sometimes the fall kills you. And sometimes, when you fall, you fly.',
  'tags': ['inspiration']},
 {'author': 'Langston Hughes',
  'quote': 'Hold fast to dreams,For if dreams dieLife is a broken-winged bird,That cannot fly.',
  'tags': ['dreams', 'hope', 'inspiration', 'knowledge', 'wisdom']},
 {'author': '//]]>',
  'quote': 'And now here

In [26]:
# Making it all a function
def get_quotes(url):
  res = requests.get(url)
  soup = BeautifulSoup(res.text)

  quote_divs = soup.find_all('div', attrs={'class' : 'quote'})

  quotes = []

  for quote_div in quote_divs:
    quoteText_div = quote_div.find_next('div', attrs={'class':'quoteText'})
    
    striped = quoteText_div.text.strip()
    striped_lines = striped.split('\n')
    
    quote = striped_lines[0][1:-1]
    author = striped_lines[-1].strip()
    
    left_div = quote_div.find_next('div', attrs={'class' : 'left'})
    tags = [tag.text for tag in left_div.find_all("a")]

    quote_item = {
        'quote' : quote,
        'author' : author,
        'tags' : tags
    }

    quotes.append(quote_item)

  return quotes

In [27]:
total = []
url = url = 'https://www.goodreads.com/quotes/tag/{}?page={}'

#.extends adds to the same list, append adds it as a new item.
for i in range(1,6):
  total.extend(get_quotes(url.format('inspiration',i)))
  print('Processed',i)

Processed 1
Processed 2
Processed 3
Processed 4
Processed 5


In [28]:
total[0]

{'author': 'Eleanor Roosevelt',
 'quote': 'Do one thing every day that scares you.',
 'tags': ['attributed', 'fear', 'inspiration']}

In [29]:
len(total)
# So 150 quotes are now extracted because of our code snippet.

150